In [1]:
import traci
import os
import sys
import keyboard
import time

# --- Configuration ---
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

sumoBinary = "sumo-gui"  # Utilisez 'sumo-gui' pour voir la simulation
sumoCmd = [sumoBinary, "-c", "ff_heterogeneous.sumocfg", "--start", "--quit-on-end"]

In [2]:
def run_simulation():
    traci.start(sumoCmd)
    
    # 1. On définit la route une seule fois au début
    traci.route.add("bus_route_2", ["E0E1", "E1E2", "E2E3", "E3E4", "E4E5"])
    
    bus_count = 0
    
    while traci.simulation.getTime() <= 3600:
        traci.simulationStep() # Avancer d'une seconde (ou d'un pas)
        
        sim_time = traci.simulation.getTime()
        
        # 2. Vérifier si on doit ajouter un bus (toutes les 180s à partir de 180s)
        if sim_time >= 180 and sim_time % 180 == 0:
            bus_id = f"bus_2.{bus_count}"
            traci.vehicle.add(bus_id, "bus_route_2", typeID="bus")
            traci.vehicle.setVehicleClass(bus_id, "bus")
            print(f"Bus {bus_id} ajouté à t={sim_time}")
            bus_count += 1
            
        # Optionnel : Sortie manuelle avec la touche 'q'
        if keyboard.is_pressed('q'):
            break
            
    traci.close()

# --- Lancement ---
try:
    run_simulation()
except traci.exceptions.FatalTraCIError:
    print("La simulation a été fermée.")

Bus bus_2.0 ajouté à t=180.0
Bus bus_2.1 ajouté à t=360.0
Bus bus_2.2 ajouté à t=540.0
Bus bus_2.3 ajouté à t=720.0
Bus bus_2.4 ajouté à t=900.0
Bus bus_2.5 ajouté à t=1080.0
Bus bus_2.6 ajouté à t=1260.0
Bus bus_2.7 ajouté à t=1440.0
Bus bus_2.8 ajouté à t=1620.0
Bus bus_2.9 ajouté à t=1800.0
Bus bus_2.10 ajouté à t=1980.0
Bus bus_2.11 ajouté à t=2160.0
Bus bus_2.12 ajouté à t=2340.0
Bus bus_2.13 ajouté à t=2520.0
Bus bus_2.14 ajouté à t=2700.0
Bus bus_2.15 ajouté à t=2880.0
Bus bus_2.16 ajouté à t=3060.0
Bus bus_2.17 ajouté à t=3240.0
Bus bus_2.18 ajouté à t=3420.0
Bus bus_2.19 ajouté à t=3600.0


### KPIs analysis


In [3]:
import xml.etree.ElementTree as ET

def extract_full_kpis(tripinfo_file):
    try:
        tree = ET.parse(tripinfo_file)
        root = tree.getroot()
    except FileNotFoundError:
        print(f"Error: {tripinfo_file} not found. Ensure the simulation ran to completion.")
        return

    # Data containers
    data = {
        "bus": {"count": 0, "duration": 0, "waiting": 0, "loss": 0},
        "car": {"count": 0, "duration": 0, "waiting": 0, "loss": 0}
    }

    for trip in root.findall('tripinfo'):
        veh_id = trip.get('id')
        # Determine category (Buses start with 'bus_2.' based on your previous code)
        category = "bus" if "bus" in veh_id else "car"
        
        data[category]["count"] += 1
        data[category]["duration"] += float(trip.get('duration'))
        data[category]["waiting"] += float(trip.get('waitingTime'))
        data[category]["loss"] += float(trip.get('timeLoss'))

    print("\n" + "="*40)
    print("       SUMO KPI REPORT (REFERENCE)")
    print("="*40)

    for cat in ["bus", "car"]:
        count = data[cat]["count"]
        if count > 0:
            avg_duration = data[cat]["duration"] / count
            avg_waiting = data[cat]["waiting"] / count
            avg_loss = data[cat]["loss"] / count
            
            print(f"[{cat.upper()}S] (Total: {count})")
            print(f"  > Avg Travel Time: {avg_duration:.2f} s")
            print(f"  > Avg Waiting Time: {avg_waiting:.2f} s")
            print(f"  > Avg Time Loss: {avg_loss:.2f} s")
            print("-" * 20)
        else:
            print(f"[{cat.upper()}S] No data found.")

# You can call this at the very end of your simulation script
# extract_full_kpis("tripinfo.xml")
# Call this after traci.close()
extract_full_kpis("C:\\Users\\alera\\Desktop\\ENTPE\\MoCoM\\MOCOM_project\\results_tripinfo.xml")


       SUMO KPI REPORT (REFERENCE)
[BUSS] (Total: 18)
  > Avg Travel Time: 359.06 s
  > Avg Waiting Time: 240.17 s
  > Avg Time Loss: 287.50 s
--------------------
[CARS] (Total: 6093)
  > Avg Travel Time: 356.81 s
  > Avg Waiting Time: 238.16 s
  > Avg Time Loss: 288.93 s
--------------------


#### Corridor analysis

In [5]:
import xml.etree.ElementTree as ET

def get_stats_from_file(filename):
    """Extracts average data for each edge from the last interval of the file."""
    tree = ET.parse(filename)
    root = tree.getroot()
    
    edge_stats = {}
    # Get the last simulation interval (the one closest to 3600s)
    last_interval = root.findall('interval')[-1]
    
    for edge in last_interval.findall('edge'):
        e_id = edge.get('id')
        speed = float(edge.get('speed'))
        waiting = float(edge.get('waitingTime'))
        density = float(edge.get('density'))
        
        edge_stats[e_id] = {
            'speed': speed,
            'waiting': waiting,
            'density': density
        }
    return edge_stats

def compare_kpis(bus_file, car_file):
    bus_data = get_stats_from_file(bus_file)
    car_data = get_stats_from_file(car_file)
    
    edges = ["E0E1", "E1E2", "E2E3", "E3E4", "E4E5"]
    
    print("\n" + "="*75)
    print(f"{'EDGE':<8} | {'TYPE':<6} | {'SPEED (m/s)':<12} | {'WAITING (s)':<12} | {'DENSITY'}")
    print("="*75)
    
    for e in edges:
        # Bus Data
        b = bus_data.get(e, {'speed': 0, 'waiting': 0, 'density': 0})
        # Car Data
        c = car_data.get(e, {'speed': 0, 'waiting': 0, 'density': 0})
        
        print(f"{e:<8} | BUS  | {b['speed']:<12.2f} | {b['waiting']:<12.2f} | {b['density']:.2f}")
        print(f"{'':<8} | CAR  | {c['speed']:<12.2f} | {c['waiting']:<12.2f} | {c['density']:.2f}")
        
        # Calculate speed difference
        if c['speed'] > 0:
            diff = ((c['speed'] - b['speed']) / c['speed']) * 100
            print(f"{'':<8} | DIFF | {diff:>11.1f}% slower for BUS")
        print("-" * 75)

# Execution (Ensure files were generated by the simulation)
try:
    compare_kpis("bus_results.xml", "car_results.xml")
except Exception as e:
    print(f"Error: Make sure you ran the simulation with the new edgeData settings. Details: {e}")


EDGE     | TYPE   | SPEED (m/s)  | WAITING (s)  | DENSITY
E0E1     | BUS  | 0.75         | 8071.00      | 147.34
         | CAR  | 0.75         | 8071.00      | 147.34
         | DIFF |         0.0% slower for BUS
---------------------------------------------------------------------------
E1E2     | BUS  | 2.64         | 2685.00      | 67.04
         | CAR  | 2.64         | 2685.00      | 67.04
         | DIFF |         0.0% slower for BUS
---------------------------------------------------------------------------
E2E3     | BUS  | 1.62         | 4071.00      | 82.94
         | CAR  | 1.62         | 4071.00      | 82.94
         | DIFF |         0.0% slower for BUS
---------------------------------------------------------------------------
E3E4     | BUS  | 0.74         | 7784.00      | 148.70
         | CAR  | 0.74         | 7784.00      | 148.70
         | DIFF |         0.0% slower for BUS
---------------------------------------------------------------------------
E4E5     | BUS  |

### Testing new traffic lights configurations:

In [2]:
import traci
import os
import sys
import keyboard
import time

# --- Configuration ---
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

sumoBinary = "sumo-gui"  # Utilisez 'sumo-gui' pour voir la simulation
sumoCmd = [sumoBinary, "-c", "ff_heterogeneous.sumocfg", "--start", "--quit-on-end"]



def run_simulation():
    traci.start(sumoCmd)

    # Apply the improved fixed logic to all intersections
    for tls_id in ["E1", "E2", "E3", "E4"]:
        traci.trafficlight.setProgram(tls_id, "1")
        
    # 1. On définit la route une seule fois au début
    traci.route.add("bus_route_2", ["E0E1", "E1E2", "E2E3", "E3E4", "E4E5"])
    
    bus_count = 0
    
    while traci.simulation.getTime() <= 3600:
        traci.simulationStep() # Avancer d'une seconde (ou d'un pas)
        
        sim_time = traci.simulation.getTime()
        
        # 2. Vérifier si on doit ajouter un bus (toutes les 180s à partir de 180s)
        if sim_time >= 180 and sim_time % 180 == 0:
            bus_id = f"bus_2.{bus_count}"
            traci.vehicle.add(bus_id, "bus_route_2", typeID="bus")
            traci.vehicle.setVehicleClass(bus_id, "bus")
            print(f"Bus {bus_id} ajouté à t={sim_time}")
            bus_count += 1
            
        # Optionnel : Sortie manuelle avec la touche 'q'
        if keyboard.is_pressed('q'):
            break
            
    traci.close()

# --- Lancement ---
try:
    run_simulation()
except traci.exceptions.FatalTraCIError:
    print("La simulation a été fermée.")

Bus bus_2.0 ajouté à t=180.0
Bus bus_2.1 ajouté à t=360.0
Bus bus_2.2 ajouté à t=540.0
Bus bus_2.3 ajouté à t=720.0
Bus bus_2.4 ajouté à t=900.0
Bus bus_2.5 ajouté à t=1080.0
Bus bus_2.6 ajouté à t=1260.0
Bus bus_2.7 ajouté à t=1440.0
Bus bus_2.8 ajouté à t=1620.0
Bus bus_2.9 ajouté à t=1800.0
Bus bus_2.10 ajouté à t=1980.0
Bus bus_2.11 ajouté à t=2160.0
Bus bus_2.12 ajouté à t=2340.0
Bus bus_2.13 ajouté à t=2520.0
Bus bus_2.14 ajouté à t=2700.0
Bus bus_2.15 ajouté à t=2880.0
Bus bus_2.16 ajouté à t=3060.0
Bus bus_2.17 ajouté à t=3240.0
Bus bus_2.18 ajouté à t=3420.0
Bus bus_2.19 ajouté à t=3600.0


In [3]:
import xml.etree.ElementTree as ET

def get_stats_from_file(filename):
    """Extracts average data for each edge from the last interval of the file."""
    tree = ET.parse(filename)
    root = tree.getroot()
    
    edge_stats = {}
    # Get the last simulation interval (the one closest to 3600s)
    last_interval = root.findall('interval')[-1]
    
    for edge in last_interval.findall('edge'):
        e_id = edge.get('id')
        speed = float(edge.get('speed'))
        waiting = float(edge.get('waitingTime'))
        density = float(edge.get('density'))
        
        edge_stats[e_id] = {
            'speed': speed,
            'waiting': waiting,
            'density': density
        }
    return edge_stats

def compare_kpis(bus_file, car_file):
    bus_data = get_stats_from_file(bus_file)
    car_data = get_stats_from_file(car_file)
    
    edges = ["E0E1", "E1E2", "E2E3", "E3E4", "E4E5"]
    
    print("\n" + "="*75)
    print(f"{'EDGE':<8} | {'TYPE':<6} | {'SPEED (m/s)':<12} | {'WAITING (s)':<12} | {'DENSITY'}")
    print("="*75)
    
    for e in edges:
        # Bus Data
        b = bus_data.get(e, {'speed': 0, 'waiting': 0, 'density': 0})
        # Car Data
        c = car_data.get(e, {'speed': 0, 'waiting': 0, 'density': 0})
        
        print(f"{e:<8} | BUS  | {b['speed']:<12.2f} | {b['waiting']:<12.2f} | {b['density']:.2f}")
        print(f"{'':<8} | CAR  | {c['speed']:<12.2f} | {c['waiting']:<12.2f} | {c['density']:.2f}")
        
        # Calculate speed difference
        if c['speed'] > 0:
            diff = ((c['speed'] - b['speed']) / c['speed']) * 100
            print(f"{'':<8} | DIFF | {diff:>11.1f}% slower for BUS")
        print("-" * 75)

# Execution (Ensure files were generated by the simulation)
try:
    compare_kpis("bus_results.xml", "car_results.xml")
except Exception as e:
    print(f"Error: Make sure you ran the simulation with the new edgeData settings. Details: {e}")


EDGE     | TYPE   | SPEED (m/s)  | WAITING (s)  | DENSITY
E0E1     | BUS  | 0.06         | 24380.00     | 376.30
         | CAR  | 0.06         | 24380.00     | 376.30
         | DIFF |         0.0% slower for BUS
---------------------------------------------------------------------------
E1E2     | BUS  | 3.60         | 566.00       | 14.90
         | CAR  | 3.60         | 566.00       | 14.90
         | DIFF |         0.0% slower for BUS
---------------------------------------------------------------------------
E2E3     | BUS  | 0.43         | 8988.00      | 162.82
         | CAR  | 0.43         | 8988.00      | 162.82
         | DIFF |         0.0% slower for BUS
---------------------------------------------------------------------------
E3E4     | BUS  | 4.18         | 912.00       | 25.65
         | CAR  | 4.18         | 912.00       | 25.65
         | DIFF |         0.0% slower for BUS
---------------------------------------------------------------------------
E4E5     | BUS  |